In [1]:
# (Optional) This block is to surpress the warning on loading tensorflow package
import warnings
warnings.filterwarnings('ignore')

In [2]:
import hoomd
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda

In [3]:
graph = htf.graph_builder(128)
nlist = graph.nlist[:, :, :3]
r = tf.norm(nlist, axis=2)
# compute 1 / r while safely treating r = 0.
# pairwise energy. Double count -> divide by 2
inv_r6 = graph.safe_div(1., r**6)
p_energy = 4.0 / 2.0 * (inv_r6 * inv_r6 - inv_r6)
# sum over pairwise energy
energy = tf.reduce_sum(p_energy, axis=1)
forces = graph.compute_forces(energy)
avg_energy = graph.running_mean(tf.reduce_sum(energy, axis=0),
                                'average-energy')
graph.save(force_tensor=forces, model_directory='traj_model',
           out_nodes=[avg_energy])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Note: Backed-up traj_model previous model to traj_model/previous_model_0



In [4]:
universe = mda.Universe('test_topol.pdb', 'test_traj.trr')
# load example graph that calculates average energy

In [5]:
htf.run_from_trajectory('traj_model', universe, period=1, r_cut=25.)







Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [6]:
# get evaluated outnodes
variables = hoomd.htf.load_variables('traj_model', ['average-energy'])
print(variables['average-energy'])

INFO:tensorflow:Restoring parameters from traj_model/model-5
-29.727135
